In [38]:
import sqlite3
import pandas as pd
import shutil

In [35]:
# Step 1: Define column names for each table
Covid_rates_columns = [
    'outcome', 'month', 'MMWR week', 'Age group', 'Vaccine product',
    'Vaccinated with outcome', 'Fully vaccinated population',
    'Unvaccinated with outcome', 'Unvaccinated population', 'Crude vax IR',
    'Crude unvax IR', 'Crude IRR', 'Continuity correction'
]

trans_level_columns = [
    'state_name', 'county_name', 'fips_code', 'date',
    'cases_per_100K_7_day_count_change',
    'percent_test_results_reported_positive_last_7_days',
    'community_transmission_level'
]

# Step 2: Create empty DataFrames for structure
Covid_rates = pd.DataFrame(columns=Covid_rates_columns)
trans_level = pd.DataFrame(columns=trans_level_columns)

# Step 1: Load the CSV files into pandas DataFrames
Covid_rates = pd.read_csv("Data/clean/clean_data_1_updated.csv")
trans_level = pd.read_csv("Data/clean/clean_data_2_updated.csv")


# Step 3: Define the file path for the SQLite database
db_path = "SQL_data.db"

# Step 4: Create SQLite database and save tables
conn = sqlite3.connect("SQL_data.db")

# Save data into respective tables
Covid_rates.to_sql('Covid_rates', conn, index=False, if_exists='replace')
trans_level.to_sql('trans_level', conn, index=False, if_exists='replace')

# Close the database connection
# conn.close()
print(f"Database created and loaded with data from CSV files at {db_path}")

Database created and loaded with data from CSV files at SQL_data.db


In [36]:
# Query the tables in the database
tables = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("Tables in the database:")
print(tables)

# You can also fetch data from the tables
Covid_rates = pd.read_sql_query("SELECT * FROM Covid_rates LIMIT 5;", conn)
print("First 5 rows from Covid_rates table:")
print(Covid_rates)

trans_level = pd.read_sql_query("SELECT * FROM trans_level LIMIT 5;", conn)
print("First 5 rows from trans_level table:")
print(trans_level)

# Close the connection when done
conn.close()

Tables in the database:
          name
0  Covid_rates
1  trans_level
First 5 rows from Covid_rates table:
  outcome     month  MMWR week Age group Vaccine product  \
0    case  APR 2021     202114     12-17       all_types   
1    case  APR 2021     202114     18-29       all_types   
2    case  APR 2021     202114     30-49       all_types   
3    case  APR 2021     202114     50-64       all_types   
4    case  APR 2021     202114     65-79       all_types   

   Vaccinated with outcome  Fully vaccinated population  \
0                        8                      36887.0   
1                      674                    2543093.0   
2                     1847                    7428405.0   
3                     1558                    7487444.0   
4                     1672                   14398099.0   

   Unvaccinated with outcome  Unvaccinated population  Crude vax IR  \
0                      30785               17556462.0     21.687858   
1                      76736        

In [41]:
source = "SQL_data.db"
destination = "Data/"
shutil.move(source, destination)

'Data/SQL_data.db'